Antaget dataformat:

Local Time | Open | High | Low | Close | Volume | Name

Dollars-bar funktionen skal indeholde:

* Tilføj ny 'Mean' kolonne, som er udregnes
$$
\begin{align*}
    \text{Mean_HL} = \frac{\text{High} + \text{Low}}{2}
\end{align*}
$$

* Tilføj variablen 'Dollarbar_cap', som er $\text{DollarBar_cap} = max(\text{High}) \cdot max(\text{volume})$ skal rundes op til nærmeste million

* Tilføj variablen 'Total_transaction' som er $\text{Total_transaction}=\text{Rest_transaction}$ udenfor loop, og $\text{Total_transaction}=\text{Total_transaction}+\text{Volume}$ inde i loop.

* Tilføj variablen 'Rest_volume', som er $\text{Rest_transaction} = \text{Total_transaction} - \text{Dollarbar_cap}$, med start værdi 0, og kun når 'Total_transaction' > DollarBar_cap

* Tilføj vairablen 'Mean_OC', som er $\text{Mean_OC} = \frac{\text{Open} + \text{Close}}{2}$

* Tilføj variablen 'High_list' and 'Low_list', som skal append 'High' og 'Low' og når Dollarbar_cap er nået, så tage max og min af listen




start den førstes variable 'open' til 'open' og sæt 'close til den sidste rækkes open og close middelværdi (Mean_OC[1])
start den næste 'open' med Mean_OC[1] og close med Mean_OC[2]




Gem variable

* 'Dato' skal være slut Dato
* 'Open' fra 'Open' i starten ellers 'Mean_OC'
* 'High' og 'Low' fra HL_list, hvor der er taget min og max
* 'Close' fra 'Mean_OC'
* 'Type' fra 'type' i for loop variable 
* 'Volume' som er Dollarbar_cap


Funktionen virker nok ikke helt optimalt i det special tilfælde, hvor vi skal lave en dollarsbar på den sidste observation i aktie data'et. Som funktionen er nu, så vil den sidste dollarsbar være "rest", og her kan vi lægge under Dollarsbar_cap, men også over, hvilket jeg ikke er sikker på om det er okay.

Index driller, men ved ikke om det er vigtigt.

In [240]:
## Read data

import matplotlib
import pandas as pd
import numpy as np
import time
import os as os


os.chdir(r"C:\Users\Bruger\Documents\Google_Drev_data\SwissData")
df = pd.read_csv('IndexData.txt')
df_sub = df.iloc[0:2000000,:]

In [63]:
import math
import numpy as np


def roundup(x, k):
    return int(math.ceil(x / k)) * k


Value = []
for name in pd.unique(df['Name']):
    x = np.max(df[df['Name'] == name]['High'])*np.max(df[df['Name'] == name]['Volume'])
    print(x)
    Value = len(str(int(x/100)))

roundup_df = pd.DataFrame({'Name' : pd.unique(df['Name']), 'Value' : Value})
roundup_df

92849950018.5535
945852054564.003


,Name,Value
0,DAX,10
1,HK,10


In [33]:


def dollarbar(df, time_unit, roundup_df):
    if isinstance(df, pd.DataFrame):
        
        ## Create empty dataframe
        df_Dollarbar = pd.DataFrame({'CPHTime' : [], 'Period' : [], 'Open' : [], 'Low' : [], 'High' : [], 
                                      'Close' : [], 'Volume' : [], 'Monetary_Volume' : [], 'Name' : [], 'Type' : []})
        
        for name in pd.unique(df['Name']):
    
            df_type = df[df['Name'] == name]
            df_type = df_type.reset_index(drop=True)
            Rest_transaction = 0
            Total_transaction = 0
            Type = df_type['Type'][0]
    
            ## Empty lists
            Time_list = []
            Open_list = []
            Low_list = []
            High_list = []
            Close_list = []
            Volume_list = []
            HL_list = []
            Period_list = []
    
            ## support vairables
            Volume_help_list = []
            iter_since_dollarbar = 0
    
            ## Dollarsbar cap
            max_High = np.max(df_type['High'])
            max_Volume = np.max(df_type['Volume'])
            Dollarbar_cap = roundup(max_High * max_Volume, 10^roundup_df[roundup_df['Name'] == name]['Value'])
    
            ## First value in Open
            Open_list.append(df_type['Open'][0])
    
            for n in np.arange(df_type.shape[0]):        
            
                Time = df_type['CPHTime'][n]
                Open = df_type['Open'][n]
                High = df_type['High'][n]
                Low = df_type['Low'][n]
                Close = df_type['Close'][n]
                Volume = df_type['Volume'][n]
        
                Mean_HL = np.mean([High, Low])
                HL_list.append(High)
                HL_list.append(Low)
        
                Total_transaction = Total_transaction + Mean_HL*Volume 
                Mean_OC = np.mean([Open, Close])
        
                Volume_help_list.append(Volume)
        
                iter_since_dollarbar += 1
            
                if n % 10000 == 0:
                    print(n)
        
                if Dollarbar_cap < Total_transaction:   #and df_type.shape[0] > n
            
                    Rest_transaction = Total_transaction - Dollarbar_cap
            
                    max_High = np.max(HL_list)
                    min_Low = np.min(HL_list)
            
                    p = (Dollarbar_cap - Total_transaction + Mean_HL*Volume)/(Mean_HL*Volume) ## andel af volume som skal med i denne dollarsbar
                    Volume_help_list[-1] = Volume*p
                    Sum_volume = np.sum(Volume_help_list)
            
                    ## Saving dollarbar values
                    Time_list.append(Time)
                    Period_list.append(iter_since_dollarbar*time_unit)
                    Low_list.append(min_Low)
                    High_list.append(max_High)
                    Close_list.append(Mean_OC)
                    Volume_list.append(Sum_volume)
                    Open_list.append(Mean_OC)
                    
                    HL_list = []
            
                    Volume_help_list = []
                    Volume_help_list.append(Volume*(1-p)) ## andel af volume som skal med i næste dollarsbar
                    
                    Total_transaction = Rest_transaction
            
                    iter_since_dollarbar = 0
            
                if df_type.shape[0] == n+1 and iter_since_dollarbar == 0:
                
                    Sum_volume = Volume * (1-p)
                
                    Close_list.append(Close)
                    Time_list.append(Time)  ## Time bliver det samme, hvilket ik er godt
                    Low_list.append(min_Low)
                    High_list.append(max_High)
                    Volume_list.append(Sum_volume)
            
                    df_dummy = pd.DataFrame({'CPHTime' : Time_list, 'Period' : Period_list, 'Open' : Open_list, 'Low' : Low_list, 'High' : High_list, 
                                     'Close' : Close_list, 'Volume' : Volume_list, 
                                     'Monetary_Volume' : Dollarbar_cap, 'Name' : name, 'Type' : Type})
                    df_Dollarbar = pd.concat([df_Dollarbar,df_dummy.iloc[:-1,:]])
            
                if df_type.shape[0] == n+1 and iter_since_dollarbar > 0:
            
                    max_High = np.max(HL_list)
                    min_Low = np.min(HL_list)
            
                    Sum_volume = np.sum(Volume_help_list)
            
                    ## Saving dollarbar values
                    Time_list.append(Time)
                    Period_list.append(iter_since_dollarbar*time_unit)
                    Low_list.append(min_Low)
                    High_list.append(max_High)
                    Close_list.append(Mean_HL)
                    Volume_list.append(Sum_volume)
            
                    df_dummy = pd.DataFrame({'CPHTime' : Time_list, 'Period' : Period_list, 'Open' : Open_list, 'Low' : Low_list, 'High' : High_list, 
                                     'Close' : Close_list, 'Volume' : Volume_list, 
                                     'Monetary_Volume' : Dollarbar_cap, 'Name' : name, 'Type' : Type})
                    df_dummy['Monetary_Volume'].iloc[-1] = Total_transaction
                    df_Dollarbar = pd.concat([df_Dollarbar,df_dummy])
        
        return df_Dollarbar
            
    else:
        print('Input need to be a Pandas DataFrame')
            




In [227]:

start=time.time()  # Start the time


## Running the function 
df_dollarbar = dollarbar(df, 1, roundup_df)
df_type = df_type.reset_index(drop=True)


# Print the time usage:
end = time.time()  # Stop the time
print(f"Time used: {(end-start):.1f} seconds")

Time used by LightGBM: 27.0 seconds


In [43]:
df_dollarbar[df_dollarbar['Name'] == 'DAX']

,CPHTime,Period,Open,Low,High,Close,Volume,Monetary_Volume,Name,Type
0,2018-02-01 10:20:00,16.0,12926.2790,12925.280,12946.780,12934.2295,7.188102e+06,9.300000e+10,DAX,Index
1,2018-02-01 10:34:00,30.0,12934.2295,12910.279,12943.280,12918.8795,7.192245e+06,9.300000e+10,DAX,Index
2,2018-02-01 10:54:00,50.0,12918.8795,12906.280,12925.780,12911.7295,7.199144e+06,9.300000e+10,DAX,Index
3,2018-02-01 11:03:00,59.0,12911.7295,12887.779,12922.480,12891.7300,7.204908e+06,9.300000e+10,DAX,Index
4,2018-02-01 11:08:00,64.0,12891.7300,12847.979,12894.980,12855.8795,7.224426e+06,9.300000e+10,DAX,Index
...,...,...,...,...,...,...,...,...,...,...
10310,2022-04-29 18:54:00,1393563.0,14047.7780,14043.749,14145.269,14141.7590,6.596340e+06,9.300000e+10,DAX,Index
10311,2022-04-29 20:13:00,1393642.0,14141.7590,14082.749,14191.399,14171.7840,6.583224e+06,9.300000e+10,DAX,Index
10312,2022-04-29 22:01:00,1393750.0,14171.7840,14048.249,14181.399,14081.6740,6.599450e+06,9.300000e+10,DAX,Index
10313,2022-04-30 01:31:00,1393960.0,14081.6740,13947.649,14083.669,13950.9380,6.630220e+06,9.300000e+10,DAX,Index


### More data

New data
* ROC-5: 5 days change of rate $\frac{\text{Close}_{new}}{\text{Close}_{5}}$
* ROC-10: 10 days change of rate $\frac{\text{Close}_{new}}{\text{Close}_{10}}$
* ROC-15: 15 days change of rate $\frac{\text{Close}_{new}}{\text{Close}_{15}}$
* ROC-20: 20 days change of rate $\frac{\text{Close}_{new}}{\text{Close}_{20}}$
* EMA-days: 10/50/200 days exponential moving average $\text{Close}_{new}\cdot\frac{\beta}{1+\text{Days}}+\text{EMA}_{old}\cdot(1-\frac{\beta}{1+\text{Days}})$

Formel used for EMA: https://www.investopedia.com/terms/e/ema.asp


In [233]:
def ROC(df, days):
    if isinstance(df, pd.DataFrame):
        final_df = pd.DataFrame()
        for name in pd.unique(df['Name']):
            
            df_type = df[df['Name'] == name]
            
            close = df_type['Close']
            ROC_df = close.pct_change(periods=days)
            
            final_df = pd.concat([final_df,ROC_df])
            
        final_df.replace(np.nan, 0, inplace = True)       
        return final_df.to_numpy().reshape(-1)
        
    else:
        print('Input need to be a Pandas DataFrame')

        

ROC_5 = ROC(df_dollarbar, 5)
ROC_10 = ROC(df_dollarbar, 10)
ROC_15 = ROC(df_dollarbar, 15)
ROC_20 = ROC(df_dollarbar, 20)
df_dummy = pd.DataFrame({'ROC-5' : ROC_5, 'ROC-10' : ROC_10,
                        'ROC-15' : ROC_15, 'ROC-20' : ROC_20})
df_Dollarbar = pd.concat([df_Dollarbar,df_dummy])
  

def EMA(df, days, smoothing = 2):
    if isinstance(df, pd.DataFrame):
        final_list = []
        k = smoothing / (days + 1)
        
        for name in pd.unique(df['Name']):
            
            df_type = df[df['Name'] == name]
            df_type = df_type.reset_index(drop=True)
            
            EMA_list = [0]*days
            SMA = np.mean(df_type['Close'][0:days])
            EMA = SMA # Our first previous EMA
            
            for n in np.arange(days,df_type.shape[0]):
                
                close = df_type['Close'][n]
                EMA = close * k + EMA * (1-k)
                EMA_list.append(EMA)
                
                if n % 100000 == 0:
                    print(n)
            
            final_list = final_list + EMA_list
        
        return final_list
        
    else:
        print('Input need to be a Pandas DataFrame')

        

EMA_10 = EMA(df_dollarbar, 10)
EMA_50 = EMA(df_dollarbar, 15)
EMA_200 = EMA(df_dollarbar, 20)
df_dummy = pd.DataFrame({'EMA-10' : EMA_10, 'EMA-50' : EMA_50, 'EMA-200' : EMA_200})
df_Dollarbar = pd.concat([df_Dollarbar,df_dummy])

In [238]:
df

,Local time,Open,High,Low,Close,Volume,Name,Type,CET,Year,Month,Day,Hour,Minute
0,02.01.2018 08:05:00.000 GMT+0100,12926.279,12930.979,12925.280,12930.779,689999.9976,DAX,Index,2018-02-01 08:05:00,2018,2,1,8,5
1,02.01.2018 08:06:00.000 GMT+0100,12930.479,12940.280,12929.780,12936.979,879999.9952,DAX,Index,2018-02-01 08:06:00,2018,2,1,8,6
2,02.01.2018 08:07:00.000 GMT+0100,12936.780,12943.279,12936.780,12942.480,540000.0215,DAX,Index,2018-02-01 08:07:00,2018,2,1,8,7
3,02.01.2018 08:08:00.000 GMT+0100,12942.780,12946.780,12942.279,12942.980,620000.0048,DAX,Index,2018-02-01 08:08:00,2018,2,1,8,8
4,02.01.2018 08:09:00.000 GMT+0100,12942.480,12942.480,12939.280,12940.280,479999.9893,DAX,Index,2018-02-01 08:09:00,2018,2,1,8,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6473566,29.04.2022 21:55:00.000 GMT+0200,7450.298,7450.298,7443.031,7444.805,70799.9989,DOWJONES,Index,2022-04-29 23:55:00,2022,4,29,23,55
6473567,29.04.2022 21:56:00.000 GMT+0200,7444.570,7447.563,7437.062,7441.802,41999.9994,DOWJONES,Index,2022-04-29 23:56:00,2022,4,29,23,56
6473568,29.04.2022 21:57:00.000 GMT+0200,7442.055,7442.055,7437.056,7437.580,35199.9998,DOWJONES,Index,2022-04-29 23:57:00,2022,4,29,23,57
6473569,29.04.2022 21:58:00.000 GMT+0200,7437.053,7445.067,7437.053,7441.543,28799.9995,DOWJONES,Index,2022-04-29 23:58:00,2022,4,29,23,58


### Saving data

In [ ]:
df_dollarbar.to_csv('DollarbarData.txt', index = False, header=True)